In [79]:
import geopandas as gpd
import netCDF4 as netcdf
import pandas as pd
from concorde.tools import get_list
from tqdm import tqdm
from kalpana.export import getDates
import pickle

In [2]:
pathin = r'/media/tacuevas/Extreme SSD/batch02'
files = sorted(get_list(pathin, ends='fort.63.nc'))

In [3]:
bbox_file = r'../gis/gpkg/bounding_box_max_flooding.gpkg'
bbox = gpd.read_file(bbox_file)
bbox = list(bbox.geometry.iloc[0].boundary.coords)

In [4]:
maxWLatNC = pd.read_pickle(r'../models/adcirc/concorde/batch02/_postprocessing/max_water_level_NC.pkl')

In [72]:
dct = {}
for j, i in tqdm(enumerate(maxWLatNC.index)):
    f63 = netcdf.Dataset(files[j])
    xvert = list(f63['x'][:].data).index(maxWLatNC.loc[i, 'x'])
    yvert = list(f63['y'][:].data).index(maxWLatNC.loc[i, 'y'])
    ts = f63['zeta'][:, xvert].data
    dates = getDates(files[j])
    dates['zeta'] = ts
    dct[i] = dates

In [80]:
with open(r'../models/adcirc/concorde/batch02/_postprocessing/time_series_water_level_at_max_NC.pkl', 'wb') as handle:
    pickle.dump(dct, handle, protocol=pickle.HIGHEST_PROTOCOL)